In [2]:
import torch
from torch import optim
import torch.nn as nn
import matplotlib.pyplot as plt 
import numpy as np
import random, pdb, math

class VoiceTechniqueClassifier:
    def __init__(self, config, spmel_params):
        """ initialise configurations"""
        self.config = config
        self.device = torch.device(f'cuda:{self.config.which_cuda}' if torch.cuda.is_available() else 'cpu')
        melsteps_per_second = spmel_params['sr'] / spmel_params['hop_size']
        self.window_size = math.ceil(config.chunk_seconds * melsteps_per_second)

        if config.is_wilkins == True:
            self.model = models.WilkinsAudioCNN(config)
        else:
            self.model = models.Luo2019AsIs(config, spmel_params)

        self.optimizer = optim.Adam(self.model.parameters(), lr=config.lr, weight_decay=config.reg)
        if self.config.load_ckpt != '': 
            g_checkpoint = torch.load(self.config.load_ckpt)
            self.model.load_state_dict(g_checkpoint['model_state_dict'])
            self.optimizer.load_state_dict(g_checkpoint['self.optimizer_state_dict'])
            # pdb.set_trace()
            # fixes tensors on different devices error
            # https://github.com/pytorch/pytorch/issues/2830
            for state in self.optimizer.state.values():
                for k, v in state.items():
                    if isinstance(v, torch.Tensor):
                        state[k] = v.cuda(self.config.which_cuda)
            self.previous_ckpt_iters = g_checkpoint['epoch']
        else:
            self.previous_ckpt_iters = 0 
        self.model.to(self.device)

    def infer(self, epoch, loader, history_list, writer, examples_per_epoch, mode):

        def batch_iterate():

            print(f'=====> {split_name}: ')
            accum_loss = 0 
            accum_corrects = 0 
            for batch_num, (x_data, y_data, singer_id)  in enumerate(loader):

                x_data = x_data.to(self.device, dtype=torch.float)
                y_data = y_data.to(self.device)

                np.save('x_data_numpy', x_data.cpu().detach().numpy())
                np.save('y_data_numpy', y_data.cpu().detach().numpy())

                #######################
    
                #Split up each example in subchunks
    
                chunk_nums = []
                for i, example in enumerate(x_data):
                    chunk_num = math.ceil(example.shape[0] / self.window_size)
                    chunk_nums.append(chunk_num)

                new_x_data_batch = []
                for i in range(len(x_data)):
                    for j in range(chunk_num):
                        offset = j * self.window_size
                        batch = x_data[j][offset : offset+self.window_size]
                        new_x_data_batch.append(batch)
                        pdb.set_trace()
                pdb.set_trace()
                new_x_data_batch = new_x_data_batch.to(self.device, dtype=torch.float)

                ######################

                tester = [1,2,3]
                prediction = self.model(new_x_data_batch, chunk_nums)
                loss = nn.functional.cross_entropy(prediction, y_data)
                _, predicted = torch.max(prediction.data, 1)
                corrects = (predicted == y_data).sum().item()
                accum_corrects += corrects
                accuracy = corrects / y_data.shape[0]
                accum_loss += loss.item()

                if mode == 'train':
                    self.optimizer.zero_grad()
                    loss.backward()
                    self.optimizer.step()

                print('Epoch {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAcc: {:.6f}'.format(
                    # inaccurate reading here if on last batch and drop_last=False
                    epoch,
                    batch_num * self.config.batch_size,
                    examples_per_epoch,
                    100. * batch_num / len(loader),
                    loss.item(),
                    accuracy)) # calculates average loss per example

                y_data = np.expand_dims(y_data.cpu(),1)
                singer_id = np.expand_dims(singer_id.cpu(),1)
                if batch_num == 0:
                    labels = np.hstack((y_data,singer_id))
                else:
                    tmp =  np.hstack((y_data,singer_id))
                    labels = np.vstack((labels, tmp))

            return labels, accum_loss, accum_corrects

        if mode == 'train':
            self.model.train()
            loss_hist=history_list[0]
            acc_hist=history_list[1]
            split_name = 'train'
            labels, accum_loss, accum_corrects = batch_iterate()
        elif mode == 'eval':
            self.model.eval()
            loss_hist=history_list[2]
            acc_hist=history_list[3]
            split_name = 'test'
            with torch.no_grad():
                labels, accum_loss, accum_corrects = batch_iterate()

        epoch_loss = accum_loss / len(loader)
        epoch_accuracy = accum_corrects / examples_per_epoch
        if self.config.is_wilkins:
            writer.add_scalar(f"Number Correct/{split_name}", accum_corrects, epoch)
            writer.add_scalar(f"Accuracy/{split_name}", epoch_accuracy, epoch)
            writer.add_scalar(f"Loss/{split_name}", epoch_loss, epoch)
            writer.add_histogram(f"layer_seq1.bias", self.model.layer_seq1[0].bias, epoch)
            writer.add_histogram(f"layer_seq1.weight", self.model.layer_seq1[0].weight, epoch)
            writer.add_histogram(f"layer_seq1.weight.grad", self.model.layer_seq1[0].weight.grad, epoch)
        else:
            writer.add_scalar(f"Number Correct/{split_name}", accum_corrects, epoch)
            writer.add_scalar(f"Accuracy/{split_name}", epoch_accuracy, epoch)
            writer.add_scalar(f"Loss/{split_name}", epoch_loss, epoch)
#            writer.add_histogram(f"enc_convs_conv_layer1", self.model.enc_convs[0][0].bias, epoch)
#            writer.add_histogram(f"enc_convs_conv_layer1.weight", self.model.enc_convs[0][0].weight, epoch)
#            writer.add_histogram(f"enc_convs_conv_layer1.weight.grad", self.model.enc_convs[0][0].weight.grad, epoch) 
        print()
        print('Epoch {} Loss: {:.4f}, Acc: {:.4f}'.format(epoch, epoch_loss, epoch_accuracy))
        loss_hist.append(epoch_loss)
        acc_hist.append(epoch_accuracy)
        save_path = './results/' +self.config.file_name +'/' +str(epoch) +'Epoch_checkpoint.pth.tar'
        self.save_checkpoints(epoch, epoch_loss, epoch_accuracy, save_path)

        return labels

    def save_checkpoints(self, epoch, loss, accuracy, save_path):
        if epoch == self.config.epochs or epoch % self.config.ckpt_freq == 0:
            print('saving model')
            checkpoint = {'model_state_dict' : self.model.state_dict(),
                'optimizer_state_dict': self.optimizer.state_dict(),
                'epoch': epoch,
                'loss': loss,
                'accuracy': accuracy}
            torch.save(checkpoint, save_path)

In [3]:
import models
from utils import saveHistory
from data import pathSpecDataset, audioSnippetDataset
import pickle, argparse, re, pdb, json, yaml, random, time, os, csv 
import numpy as np
from tqdm import tqdm
import torch
from torch import optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler

In [9]:
config = pickle.load(open('config_object','rb'))

In [12]:
torch.manual_seed(1)
device = torch.device(f"cuda:{config.which_cuda}" if config.cuda else "cpu")
seconds = time.time()

results_dir = './results'
if not os.path.exists(results_dir):
    os.mkdir(results_dir)
    
file_name_dir = './results/' +config.file_name
if not os.path.exists(file_name_dir):
    os.mkdir(file_name_dir)
results_csv='./results/' +config.file_name +'/RandomSearchReport.csv'


history_list=[[], [], [], []]

"""For classification technique we can do a train/test split like in Wilkins, of 0.75"""
# look at the amount of files in folder
# randomly choose which singers will be used in training set
# sort directory melspec by name and generate indices for training set and test set
#if config.split_by == 'singer':
print('here1', time.time() - seconds)
m_list = ['m1_','m2_','m3_','m4_','m5_','m6_','m7_','m8_','m9_','m10_','m11_']
f_list = ['f1_','f2_','f3_','f4_','f5_','f6_','f7_','f8_','f9_']
random.shuffle(m_list)
random.shuffle(f_list)
train_m_list, test_m_list = (m_list[:-3],m_list[-3:])
train_f_list, test_f_list = (f_list[:-2],f_list[-2:])

train_list = train_m_list + train_f_list
test_list = test_m_list + test_f_list
#print('test_list', test_list)
""" its too complex to write a universal specPathDataset with different subfolder structures.
More ignostic to automatically upload from one shallow directory, and sort from there using the filename analysis.
Make sure the dataset is fed data in same order as sorted fileList"""

with open('spmel_desilenced/spmel_params.yaml') as File:
    spmel_params = yaml.load(File, Loader=yaml.FullLoader)

here1 0.00024008750915527344


In [32]:
from torch.utils.data import Dataset, DataLoader
import soundfile as sf
import numpy as np
import os, pdb, pickle, random, math, torch

from multiprocessing import Process, Manager


class pathSpecDataset(Dataset):
    """Dataset class for using a path to spec folders,
        path for labels,
        generates random windowed subspec examples,
        associated labels,
        optional conditioning."""
    def __init__(self, config, spmel_params):
        """Initialize and preprocess the dataset."""
        self.spmel_dir = config.data_dir
        melsteps_per_second = spmel_params['sr'] / spmel_params['hop_size']
        self.window_size = math.ceil(config.chunk_seconds * melsteps_per_second)
    
        style_names = ['belt','lip_trill','straight','vocal_fry','vibrato','breathy']
        singer_names = ['m1_','m2_','m3_','m4_','m5_','m6_','m7_','m8_','m9_','m10_','m11_','f1_','f2_','f3_','f4_','f5_','f6_','f7_','f8_','f9_']
        #self.one_hot_array = np.eye(len(class_names))[np.arange(len(class_names))]
        dir_name, _, fileList = next(os.walk(self.spmel_dir))
        fileList = sorted(fileList)
        dataset = []
        for file_name in fileList:
            if file_name.endswith('.npy'):
                file_path = os.path.join(dir_name, file_name)
                spmel = np.load(file_path)
                print(file_path, spmel)
                for style_idx, style_name in enumerate(style_names):
                    if style_name in file_name:
                        for singer_idx, singer_name in enumerate(singer_names):
                            if singer_name in file_name:
                                dataset.append((spmel, style_idx, singer_idx))
                                break
                        break

        self.dataset = dataset
        self.num_specs = len(dataset)
        pdb.set_trace

    """__getitem__ selects a speaker and chooses a random subset of data (in this case
    an utterance) and randomly crops that data. It also selects the corresponding speaker
    embedding and loads that up. It will now also get corresponding pitch contour for such a file"""
    def __getitem__(self, index):
        # pick a random speaker
        dataset = self.dataset
        # spkr_data is literally a list of skpr_id, emb, and utterances from a single speaker
        spmel, style_idx, singer_idx  = dataset[index]
        # pick random spmel_chunk with random crop


        """old incorrect way of feeding data to network"""
############################################################################
#
#        if spmel.shape[0] < self.window_size:
#            len_pad = self.window_size - spmel.shape[0]
#            spmel_chunk = np.pad(spmel, ((0,len_pad),(0,0)), 'constant')
#            #pitch = np.pad(pitch_info, ((0,len_pad),(0,0)), 'constant')
#        elif spmel.shape[0] > self.window_size:
#            left = np.random.randint(spmel.shape[0]-self.window_size)
#            spmel_chunk = spmel[left:left+self.window_size, :]
#            #pitch = pitch_info[left:left+self.window_size, :]
#        else:
#            spmel_chunk = spmel
#            #pitch = pitch_info
# 
#        return spmel_chunk, style_idx, singer_idx
############################################################################

        return spmel, style_idx, singer_idx

    def __len__(self):
        """Return the number of spkrs."""
        return self.num_specs

In [33]:
if config.is_wilkins == True:
    dataset = audioSnippetDataset(config)
    model = models.WilkinsAudioCNN(config)
    fileList = pickle.load(open(config.data_dir, 'rb'))
else:
    _, _, fileList = next(os.walk('./spmel_desilenced'))
    dataset = pathSpecDataset(config, spmel_params)
    model = models.Luo2019AsIs(config, spmel_params)

./spmel_desilenced/f1_arpeggios_belt_c_a.npy [[0.25476113 0.29229352 0.4135683  ... 0.         0.         0.        ]
 [0.3694387  0.35387635 0.40471697 ... 0.         0.         0.        ]
 [0.4066043  0.3969838  0.42049694 ... 0.         0.         0.        ]
 ...
 [0.18086022 0.14914557 0.24672641 ... 0.         0.         0.        ]
 [0.31352904 0.27560836 0.27100995 ... 0.         0.         0.        ]
 [0.4453155  0.4337174  0.4451427  ... 0.         0.         0.        ]]
./spmel_desilenced/f1_arpeggios_belt_c_e.npy [[0.25476113 0.29229352 0.4135683  ... 0.         0.         0.        ]
 [0.3694387  0.35387635 0.40471697 ... 0.         0.         0.        ]
 [0.4066043  0.3969838  0.42049694 ... 0.         0.         0.        ]
 ...
 [0.18086022 0.14914557 0.24672641 ... 0.         0.         0.        ]
 [0.31352904 0.27560836 0.27100995 ... 0.         0.         0.        ]
 [0.4453155  0.4337174  0.4451427  ... 0.         0.         0.        ]]
./spmel_desilenced/f1_

./spmel_desilenced/f2_scales_lip_trill_u.npy [[0.25476113 0.29229352 0.4135683  ... 0.         0.         0.        ]
 [0.3694387  0.35387635 0.40471697 ... 0.         0.         0.        ]
 [0.4066043  0.3969838  0.42049694 ... 0.         0.         0.        ]
 ...
 [0.18086022 0.14914557 0.24672641 ... 0.         0.         0.        ]
 [0.31352904 0.27560836 0.27100995 ... 0.         0.         0.        ]
 [0.4453155  0.4337174  0.4451427  ... 0.         0.         0.        ]]
./spmel_desilenced/f2_scales_straight_a.npy [[0.25476113 0.29229352 0.4135683  ... 0.         0.         0.        ]
 [0.3694387  0.35387635 0.40471697 ... 0.         0.         0.        ]
 [0.4066043  0.3969838  0.42049694 ... 0.         0.         0.        ]
 ...
 [0.18086022 0.14914557 0.24672641 ... 0.         0.         0.        ]
 [0.31352904 0.27560836 0.27100995 ... 0.         0.         0.        ]
 [0.4453155  0.4337174  0.4451427  ... 0.         0.         0.        ]]
./spmel_desilenced/f2_s

./spmel_desilenced/f4_arpeggios_straight_a.npy [[0.25476113 0.29229352 0.4135683  ... 0.         0.         0.        ]
 [0.3694387  0.35387635 0.40471697 ... 0.         0.         0.        ]
 [0.4066043  0.3969838  0.42049694 ... 0.         0.         0.        ]
 ...
 [0.18086022 0.14914557 0.24672641 ... 0.         0.         0.        ]
 [0.31352904 0.27560836 0.27100995 ... 0.         0.         0.        ]
 [0.4453155  0.4337174  0.4451427  ... 0.         0.         0.        ]]
./spmel_desilenced/f4_arpeggios_straight_e.npy [[0.25476113 0.29229352 0.4135683  ... 0.         0.         0.        ]
 [0.3694387  0.35387635 0.40471697 ... 0.         0.         0.        ]
 [0.4066043  0.3969838  0.42049694 ... 0.         0.         0.        ]
 ...
 [0.18086022 0.14914557 0.24672641 ... 0.         0.         0.        ]
 [0.31352904 0.27560836 0.27100995 ... 0.         0.         0.        ]
 [0.4453155  0.4337174  0.4451427  ... 0.         0.         0.        ]]
./spmel_desilenced

./spmel_desilenced/f6_arpeggios_belt_2_a.npy [[0.25476113 0.29229352 0.4135683  ... 0.         0.         0.        ]
 [0.3694387  0.35387635 0.40471697 ... 0.         0.         0.        ]
 [0.4066043  0.3969838  0.42049694 ... 0.         0.         0.        ]
 ...
 [0.18086022 0.14914557 0.24672641 ... 0.         0.         0.        ]
 [0.31352904 0.27560836 0.27100995 ... 0.         0.         0.        ]
 [0.4453155  0.4337174  0.4451427  ... 0.         0.         0.        ]]
./spmel_desilenced/f6_arpeggios_belt_2_e.npy [[0.25476113 0.29229352 0.4135683  ... 0.         0.         0.        ]
 [0.3694387  0.35387635 0.40471697 ... 0.         0.         0.        ]
 [0.4066043  0.3969838  0.42049694 ... 0.         0.         0.        ]
 ...
 [0.18086022 0.14914557 0.24672641 ... 0.         0.         0.        ]
 [0.31352904 0.27560836 0.27100995 ... 0.         0.         0.        ]
 [0.4453155  0.4337174  0.4451427  ... 0.         0.         0.        ]]
./spmel_desilenced/f6_

./spmel_desilenced/f7_scales_vocal_fry_o.npy [[0.25476113 0.29229352 0.4135683  ... 0.         0.         0.        ]
 [0.3694387  0.35387635 0.40471697 ... 0.         0.         0.        ]
 [0.4066043  0.3969838  0.42049694 ... 0.         0.         0.        ]
 ...
 [0.18086022 0.14914557 0.24672641 ... 0.         0.         0.        ]
 [0.31352904 0.27560836 0.27100995 ... 0.         0.         0.        ]
 [0.4453155  0.4337174  0.4451427  ... 0.         0.         0.        ]]
./spmel_desilenced/f7_scales_vocal_fry_u.npy [[0.25476113 0.29229352 0.4135683  ... 0.         0.         0.        ]
 [0.3694387  0.35387635 0.40471697 ... 0.         0.         0.        ]
 [0.4066043  0.3969838  0.42049694 ... 0.         0.         0.        ]
 ...
 [0.18086022 0.14914557 0.24672641 ... 0.         0.         0.        ]
 [0.31352904 0.27560836 0.27100995 ... 0.         0.         0.        ]
 [0.4453155  0.4337174  0.4451427  ... 0.         0.         0.        ]]
./spmel_desilenced/f8_

...

./spmel_desilenced/m2_scales_breathy_i.npy [[0.25476113 0.29229352 0.4135683  ... 0.         0.         0.        ]
 [0.3694387  0.35387635 0.40471697 ... 0.         0.         0.        ]
 [0.4066043  0.3969838  0.42049694 ... 0.         0.         0.        ]
 ...
 [0.18086022 0.14914557 0.24672641 ... 0.         0.         0.        ]
 [0.31352904 0.27560836 0.27100995 ... 0.         0.         0.        ]
 [0.4453155  0.4337174  0.4451427  ... 0.         0.         0.        ]]
./spmel_desilenced/m2_scales_breathy_o.npy [[0.25476113 0.29229352 0.4135683  ... 0.         0.         0.        ]
 [0.3694387  0.35387635 0.40471697 ... 0.         0.         0.        ]
 [0.4066043  0.3969838  0.42049694 ... 0.         0.         0.        ]
 ...
 [0.18086022 0.14914557 0.24672641 ... 0.         0.         0.        ]
 [0.31352904 0.27560836 0.27100995 ... 0.         0.         0.        ]
 [0.4453155  0.4337174  0.4451427  ... 0.         0.         0.        ]]
./spmel_desilenced/m2_scal

./spmel_desilenced/m4_arpeggios_vocal_fry_a.npy [[0.25476113 0.29229352 0.4135683  ... 0.         0.         0.        ]
 [0.3694387  0.35387635 0.40471697 ... 0.         0.         0.        ]
 [0.4066043  0.3969838  0.42049694 ... 0.         0.         0.        ]
 ...
 [0.18086022 0.14914557 0.24672641 ... 0.         0.         0.        ]
 [0.31352904 0.27560836 0.27100995 ... 0.         0.         0.        ]
 [0.4453155  0.4337174  0.4451427  ... 0.         0.         0.        ]]
./spmel_desilenced/m4_arpeggios_vocal_fry_e.npy [[0.25476113 0.29229352 0.4135683  ... 0.         0.         0.        ]
 [0.3694387  0.35387635 0.40471697 ... 0.         0.         0.        ]
 [0.4066043  0.3969838  0.42049694 ... 0.         0.         0.        ]
 ...
 [0.18086022 0.14914557 0.24672641 ... 0.         0.         0.        ]
 [0.31352904 0.27560836 0.27100995 ... 0.         0.         0.        ]
 [0.4453155  0.4337174  0.4451427  ... 0.         0.         0.        ]]
./spmel_desilenc

./spmel_desilenced/m6_arpeggios_straight_i.npy [[0.25476113 0.29229352 0.4135683  ... 0.         0.         0.        ]
 [0.3694387  0.35387635 0.40471697 ... 0.         0.         0.        ]
 [0.4066043  0.3969838  0.42049694 ... 0.         0.         0.        ]
 ...
 [0.18086022 0.14914557 0.24672641 ... 0.         0.         0.        ]
 [0.31352904 0.27560836 0.27100995 ... 0.         0.         0.        ]
 [0.4453155  0.4337174  0.4451427  ... 0.         0.         0.        ]]
./spmel_desilenced/m6_arpeggios_straight_o.npy [[0.25476113 0.29229352 0.4135683  ... 0.         0.         0.        ]
 [0.3694387  0.35387635 0.40471697 ... 0.         0.         0.        ]
 [0.4066043  0.3969838  0.42049694 ... 0.         0.         0.        ]
 ...
 [0.18086022 0.14914557 0.24672641 ... 0.         0.         0.        ]
 [0.31352904 0.27560836 0.27100995 ... 0.         0.         0.        ]
 [0.4453155  0.4337174  0.4451427  ... 0.         0.         0.        ]]
./spmel_desilenced

./spmel_desilenced/m7_scales_vibrato_e.npy [[0.25476113 0.29229352 0.4135683  ... 0.         0.         0.        ]
 [0.3694387  0.35387635 0.40471697 ... 0.         0.         0.        ]
 [0.4066043  0.3969838  0.42049694 ... 0.         0.         0.        ]
 ...
 [0.18086022 0.14914557 0.24672641 ... 0.         0.         0.        ]
 [0.31352904 0.27560836 0.27100995 ... 0.         0.         0.        ]
 [0.4453155  0.4337174  0.4451427  ... 0.         0.         0.        ]]
./spmel_desilenced/m7_scales_vibrato_i.npy [[0.25476113 0.29229352 0.4135683  ... 0.         0.         0.        ]
 [0.3694387  0.35387635 0.40471697 ... 0.         0.         0.        ]
 [0.4066043  0.3969838  0.42049694 ... 0.         0.         0.        ]
 ...
 [0.18086022 0.14914557 0.24672641 ... 0.         0.         0.        ]
 [0.31352904 0.27560836 0.27100995 ... 0.         0.         0.        ]
 [0.4453155  0.4337174  0.4451427  ... 0.         0.         0.        ]]
./spmel_desilenced/m7_scal

./spmel_desilenced/m9_arpeggios_breathy_u.npy [[0.25476113 0.29229352 0.4135683  ... 0.         0.         0.        ]
 [0.3694387  0.35387635 0.40471697 ... 0.         0.         0.        ]
 [0.4066043  0.3969838  0.42049694 ... 0.         0.         0.        ]
 ...
 [0.18086022 0.14914557 0.24672641 ... 0.         0.         0.        ]
 [0.31352904 0.27560836 0.27100995 ... 0.         0.         0.        ]
 [0.4453155  0.4337174  0.4451427  ... 0.         0.         0.        ]]
./spmel_desilenced/m9_arpeggios_lip_trill_a.npy [[0.25476113 0.29229352 0.4135683  ... 0.         0.         0.        ]
 [0.3694387  0.35387635 0.40471697 ... 0.         0.         0.        ]
 [0.4066043  0.3969838  0.42049694 ... 0.         0.         0.        ]
 ...
 [0.18086022 0.14914557 0.24672641 ... 0.         0.         0.        ]
 [0.31352904 0.27560836 0.27100995 ... 0.         0.         0.        ]
 [0.4453155  0.4337174  0.4451427  ... 0.         0.         0.        ]]
./spmel_desilenced

In [21]:
len(dataset)

1182

In [25]:
import matplotlib.pyplot as plt

for i in range(300, 330):
    print(dataset[i])
#     spmel, style_idx, singer_idx = dataset[i]
#     spmel = np.rot90(spmel)
#     plt.imshow(spmel, cmap='gray')
#     plt.show()

(array([[0.25476113, 0.29229352, 0.4135683 , ..., 0.        , 0.        ,
        0.        ],
       [0.3694387 , 0.35387635, 0.40471697, ..., 0.        , 0.        ,
        0.        ],
       [0.4066043 , 0.3969838 , 0.42049694, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.18086022, 0.14914557, 0.24672641, ..., 0.        , 0.        ,
        0.        ],
       [0.31352904, 0.27560836, 0.27100995, ..., 0.        , 0.        ,
        0.        ],
       [0.4453155 , 0.4337174 , 0.4451427 , ..., 0.        , 0.        ,
        0.        ]], dtype=float32), 0, 16)
(array([[0.25476113, 0.29229352, 0.4135683 , ..., 0.        , 0.        ,
        0.        ],
       [0.3694387 , 0.35387635, 0.40471697, ..., 0.        , 0.        ,
        0.        ],
       [0.4066043 , 0.3969838 , 0.42049694, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.18086022, 0.14914557, 0.24672641, ..., 0.        , 0.        ,
        0.        ],
       [0.

In [ ]:
file_path_list = sorted(fileList)
fileList = [os.path.basename(x) for x in file_path_list]
train_indices_list = []
test_indices_list = []

for fileName_idx, fileName in enumerate(fileList):
    for substring in train_list:
        if substring in fileName:
            train_indices_list.append(fileName_idx)
    for substring in test_list:
        if substring in fileName:
            test_indices_list.append(fileName_idx)
if config.short==True:
    train_indices_list = train_indices_list[:8]
    test_indices_list = test_indices_list[:4]
    config.batch_size = 2

writer = SummaryWriter(comment = '_' +config.file_name)

epoch_labels = []
"""https://stackoverflow.com/questions/50544730/how-do-i-split-a-custom-dataset-into-training-and-test-datasets"""
train_sampler = SubsetRandomSampler(train_indices_list)
test_sampler = SubsetRandomSampler(test_indices_list)
train_loader = DataLoader(dataset, batch_size=config.batch_size, sampler=train_sampler, shuffle=False, drop_last=False)
test_loader = DataLoader(dataset, batch_size=config.batch_size, sampler=test_sampler, shuffle=False, drop_last=False)
vt_classer = VoiceTechniqueClassifier(config, spmel_params)

#example_data, example_targets, ex_singer_ids = iter(test_loader).next()
#writer.add_graph(model, example_data.float())

if config.load_ckpt != '':

if config.load_ckpt != '':
    previous_epochs = int(re.findall('\d+', config.load_ckpt)[0])
else: previous_epochs = 0
for epoch in range(previous_epochs+1, previous_epochs+config.epochs+1):
    # history_list gets extended while inside these functions
    train_labels = vt_classer.infer(epoch, train_loader, history_list, writer, len(train_indices_list), 'train')
    val_labels = vt_classer.infer(epoch, test_loader, history_list, writer, len(test_indices_list), 'eval')
    epoch_labels.append((train_labels, val_labels))
    writer.flush()

writer.close()
# model_file_name = os.path.basename(__file__)[:-3]

saveHistory(history_list, file_name_dir, string_config, epoch_labels)
#invalid terms just
bestLoss = history_list[0][0]

for idx, loss in enumerate(history_list[0]):
    if loss <= bestLoss:
        bestLoss=loss
        bestEpoch=idx

bestAcc = history_list[3][bestEpoch]
